# Arrays

### Count Number of Bad Pairs

You are given a 0-indexed integer array nums. A pair of indices (i, j) is a bad pair if i < j and j - i != nums[j] - nums[i].

Return the total number of bad pairs in nums.

 

Example 1:

Input: nums = [4,1,3,3]

Output: 5

Explanation: The pair (0, 1) is a bad pair since 1 - 0 != 1 - 4.

The pair (0, 2) is a bad pair since 2 - 0 != 3 - 4, 2 != -1.

The pair (0, 3) is a bad pair since 3 - 0 != 3 - 4, 3 != -1.

The pair (1, 2) is a bad pair since 2 - 1 != 3 - 1, 1 != 2.

The pair (2, 3) is a bad pair since 3 - 2 != 3 - 3, 1 != 0.

There are a total of 5 bad pairs, so we return 5.

In [57]:
def count_bad_pairs(nums):
    # Total number of pairs = n choose 2.
    n = len(nums)
    total_pairs = n * (n - 1) // 2

    # Count good pairs: j - i == nums[j] - nums[i] <==> (i - nums[i]) == (j - nums[j])
    diff_count = {}
    for i, num in enumerate(nums):
        diff = i - num 
        diff_count[diff] = diff_count.get(diff, 0) + 1
        # Thus, each index contributes a value diff = i - nums[i]. If two indices share the same diff, then the pair formed by these indices is good.

    print(diff_count)
    good_pairs = 0
    for count in diff_count.values():
        # For each group of indices with the same diff, count the pairs.
        good_pairs += count * (count - 1) // 2

    # Bad pairs are the total pairs minus the good pairs.
    return total_pairs - good_pairs

# Example usage:
if __name__ == "__main__":
    nums = [4, 1, 3, 3]
    print("Total number of bad pairs:", count_bad_pairs(nums))


{-4: 1, 0: 2, -1: 1}
Total number of bad pairs: 5


### XOR Queries of a Subarray

In [ ]:
queries = [[0,1],[1,2],[0,3],[3,3]]

In [78]:
def xorQueries(arr, queries):
    # Precompute the prefix XOR: prefix[i] is XOR of arr[0] to arr[i-1].
    n = len(arr)
    prefix = [0] * (n + 1)
    for i in range(n):
        print(i, i+1)
        prefix[i + 1] = prefix[i] ^ arr[i]
    
    # For each query [left, right], the result is prefix[right+1] XOR prefix[left].
    print(prefix)
    answer = []
    for left, right in queries:
        answer.append(prefix[right + 1] ^ prefix[left])
    return answer

# Example usage:
if __name__ == "__main__":
    arr = [1, 3, 4, 8]
    queries = [[0, 1], [1, 2], [0, 3], [3, 3]]
    print("XOR queries result:", xorQueries(arr, queries))
    # Expected output: [2, 7, 14, 8]

0 1
1 2
2 3
3 4
[0, 1, 2, 6, 14]
XOR queries result: [2, 7, 14, 8]


### Maximum and Minimum Sums of at Most Size K Subsequences Medium

Here s is the size of our subsequence, n is the size of nums and n−i−1 are the number of items larger than the current number in our sorted array (it's just the length of the array minus the current index). This formula is the entire solution to this problem, it computes the total contribution of each number for every size but only for the minimum. Flip it and you can compute all contributions for the maximum. However it's a little slow, given it operates in O(nk) time.

We optimize our solution with dynamic programming, by working backwards. We can only make 1 subsequence with the last number as the minimum and the first number as the maximum, so we start with quantity = 1 in our code. Then for each subsequent number we double the number of subsequences we can make up until k.

Let's revisit our code now. total_sums += quantity * (nums[i] + nums[-i - 1]) is adding the contribution made from the maximum and minimum number. Then quantity = 2 * quantity is doing this doubling of subsequences with our given number as the min/max but after i exceeds k we have to adjust for the fact that we can't make subsequences longer than k and sutract all subsequences longer than k from the quantity hence the - comb(i, k - 1).
 

In [190]:
import itertools

for sub in itertools.combinations([1,2,3,4], 2):
    print(sub)

(1, 2)
(1, 3)
(1, 4)
(2, 3)
(2, 4)
(3, 4)


In [191]:
import math
math.comb(2, 1)

2

In [151]:
# C(n,k)= n! / k!(n−k)!

nums = [1,2,3]
k = 2

def factorial_recursive(n):
    if n == 0:
        return 1
    print(n)
    return n * factorial_recursive(n - 1)

factorial_recursive(3)
#factorial_recursive(3) / factorial_recursive(k) * (factorial_recursive(3-k))

3
2
1


6

In [192]:
MOD = 10 ** 9 + 7  
  
def max_min_sums(nums, k):  
    n = len(nums)  
    nums.sort()  # Sort the array  
  
    # Precompute factorials and inverse factorials  
    maxN = n + 1  
    fac = [1] * maxN  
    ifac = [1] * maxN  
    for i in range(1, maxN):  
        fac[i] = fac[i - 1] * i % MOD  
    ifac[-1] = pow(fac[-1], MOD - 2, MOD)  
    for i in range(maxN - 2, -1, -1):  
        ifac[i] = ifac[i + 1] * (i + 1) % MOD  
  
    def comb(n, r):  
        if r < 0 or r > n:  
            return 0  
        return fac[n] * ifac[r] % MOD * ifac[n - r] % MOD  
  
    # Precompute cumulative combinations  
    cum_comb = [0] * maxN  
    for r in range(maxN):  
        cum_comb[r] = comb(n - 1, r)  
  
    # Calculate cumulative sums for combinations up to k  
    sum_cum_comb = [0] * maxN  
    for i in range(1, maxN):  
        sum_cum_comb[i] = (sum_cum_comb[i - 1] + cum_comb[i - 1]) % MOD  
  
    total_max = 0  
    total_min = 0  
    for i in range(n):  
        # Contribution as maximum  
        count_max = sum_cum_comb[min(i + 1, k)]  
        total_max = (total_max + nums[i] * count_max) % MOD  
  
        # Contribution as minimum  
        count_min = sum_cum_comb[min(n - i, k)]  
        total_min = (total_min + nums[i] * count_min) % MOD  
  
    return (total_max + total_min) % MOD  
  
# Example usage:  
nums = [3, 1, 2]  
k = 2  
print(max_min_sums(nums, k))  # Output: 24  

28


In [90]:
from math import comb


def minMaxSums(nums, k: int) -> int:
    nums.sort()
    total_sums = 0
    quantity = 1

    print(nums)
    
    for i in range(len(nums)):
        print(nums[i], nums[-i - 1])
        total_sums += quantity * (nums[i] + nums[-i - 1])
        quantity = 2 * quantity - comb(i, k - 1)

    return total_sums % (10 ** 9 + 7)

In [91]:
nums = [1,2,3]
k = 2

minMaxSums(nums, k)

[1, 2, 3]
1 3
2 2
3 1


24

## two_sum

In [1]:
def two_sum(nums, target):
    lookup = {}
    for i, num in enumerate(nums):
        if target - num in lookup:
            return [lookup[target - num], i]
        lookup[num] = i

In [2]:
two_sum([2,7,11,15], 9)

[0, 1]

## Container With Most Water

You are given an integer array height of length n. There are n vertical lines drawn such that the two endpoints of the ith line are (i, 0) and (i, height[i]).

Find two lines that together with the x-axis form a container, such that the container contains the most water.

Return the maximum amount of water a container can store.

Notice that you may not slant the container.




In [5]:
def maxArea(height):
    left, right = 0, len(height) - 1
    answer = 0
    while left < right:
        area = (right - left) * min(height[left], height[right])
        answer = max(answer, area)
        if height[left] < height[right]:
            left += 1
        else:
            right -= 1
    return answer

print(maxArea([1,8,6,2,5,4,8,3, 7]))

49


In [100]:
n = 3
x = 4

## Merge Intervals


Given an array of intervals where intervals[i] = [starti, endi], merge all overlapping intervals, and return an array of the non-overlapping intervals that cover all the intervals in the input.


Example 1:

Input: intervals = [[1,3],[2,6],[8,10],[15,18]]
Output: [[1,6],[8,10],[15,18]]
Explanation: Since intervals [1,3] and [2,6] overlap, merge them into [1,6].

In [178]:
def merge_intervals(intervals):
    """
    Merges all overlapping intervals and returns the non-overlapping intervals.
    1) Sort intervals by start value.
    2) If the current interval overlaps with the last one in the merged list, update the end.
    3) If they don't overlap, add the current interval to the result.
    """
    if not intervals:
        return []

    # Sort by starting time
    intervals.sort(key=lambda x: x[0])
    merged = [intervals[0]]

    for i in range(1, len(intervals)):
        # Compare current interval with the last merged interval
        if intervals[i][0] <= merged[-1][1]:
            # Overlaps, so merge
            merged[-1][1] = max(merged[-1][1], intervals[i][1])
        else:
            # No overlap, so add to merged list
            merged.append(intervals[i])

    return merged

# Example usage:
if __name__ == "__main__":
    intervals = [[1,3],[2,6],[8,10],[15,18]]
    print("Merged intervals:", merge_intervals(intervals))
    # Expected output: [[1,6],[8,10],[15,18]]

Merged intervals: [[1, 6], [8, 10], [15, 18]]


In [177]:
stack

[[1, 6], [2, 10]]

## 3Sum

Given an integer array nums, return all the triplets [nums[i], nums[j], nums[k]] such that i != j, i != k, and j != k, and nums[i] + nums[j] + nums[k] == 0.

Notice that the solution set must not contain duplicate triplets.


To solve the 3Sum problem, you can use the following code:

```python
def three_sum(nums):
    nums.sort()
    result = []
    for i in range(len(nums) - 2):
        if i > 0 and nums[i] == nums[i - 1]:
            continue
        left, right = i + 1, len(nums) - 1
        while left < right:
            total = nums[i] + nums[left] + nums[right]
            if total < 0:
                left += 1
            elif total > 0:
                right -= 1
            else:
                result.append([nums[i], nums[left], nums[right]])
                while left < right and nums[left] == nums[left + 1]:
                    left += 1
                while left < right and nums[right] == nums[right - 1]:
                    right -= 1
                left += 1
                right -= 1
    return result

print(three_sum([-1, 0, 1, 2, -1, -4]))
```

## 3Sum Closest
Medium

Topics
Companies
Given an integer array nums of length n and an integer target, find three integers in nums such that the sum is closest to target.

Return the sum of the three integers.

You may assume that each input would have exactly one solution.

In [31]:
nums = [-1,2,1,-4]
target = 1

In [35]:
def three_sum_closest(nums, target):
    nums.sort()
    closest_sum = float('inf')
    for i in range(len(nums) - 2):
        left, right = i + 1, len(nums) - 1
        while left < right:
            current_sum = nums[i] + nums[left] + nums[right]
            if abs(current_sum - target) < abs(closest_sum - target):
                closest_sum = current_sum
            if current_sum < target:
                left += 1
            elif current_sum > target:
                right -= 1
            else:
                return current_sum
    return closest_sum

print(three_sum_closest(nums, target))

2


## 4Sum
Given an array nums of n integers, return an array of all the unique quadruplets [nums[a], nums[b], nums[c], nums[d]] such that:

0 <= a, b, c, d < n
a, b, c, and d are distinct.
nums[a] + nums[b] + nums[c] + nums[d] == target
You may return the answer in any order.

To solve the 4Sum problem, you can use the following code:
```python
def four_sum(nums, target):
    nums.sort()
    result = []
    for i in range(len(nums) - 3):
        if i > 0 and nums[i] == nums[i - 1]:
            continue
        for j in range(i + 1, len(nums) - 2):
            if j > i + 1 and nums[j] == nums[j - 1]:
                continue
            left, right = j + 1, len(nums) - 1
            while left < right:
                total = nums[i] + nums[j] + nums[left] + nums[right]
                if total < target:
                    left += 1
                elif total > target:
                    right -= 1
                else:
                    result.append([nums[i], nums[j], nums[left], nums[right]])
                    while left < right and nums[left] == nums[left + 1]:
                        left += 1
                    while left < right and nums[right] == nums[right - 1]:
                        right -= 1
                    left += 1
                    right -= 1
    return result

print(four_sum(nums, target))
```

## --Remove Duplicates from Sorted Array

In [11]:
def remove_duplicates(nums):
    if not nums:
        return 0

    unique_index = 0

    for i in range(1, len(nums)):
        if nums[i] != nums[unique_index]:
            unique_index += 1
            nums[unique_index] = nums[i]

    return unique_index + 1

# Example usage
nums = [0, 1, 2, 3, 4, '_', '_', '_', '_', '_']
unique_count = remove_duplicates(nums)
print(unique_count)  # Output: 5
print(nums[:unique_count])  # Output: [0, 1, 2, 3, 4]

6
[0, 1, 2, 3, 4, '_']


In [16]:
nums = [0,0,1,1,1,2,2,3,3,4]

for i, num in enumerate(nums):
    if isinstance(i, int):
        if nums.count(num) > 1:
            nums.pop(i)
            nums.append("_")

len(set(nums)) - 1, nums



(5, [0, 1, 2, 3, 4, '_', '_', '_', '_', '_'])

## Next Permutation

A permutation of an array of integers is an arrangement of its members into a sequence or linear order.

For example, for arr = [1,2,3], the following are all the permutations of arr: [1,2,3], [1,3,2], [2, 1, 3], [2, 3, 1], [3,1,2], [3,2,1].
The next permutation of an array of integers is the next lexicographically greater permutation of its integer. More formally, if all the permutations of the array are sorted in one container according to their lexicographical order, then the next permutation of that array is the permutation that follows it in the sorted container. If such arrangement is not possible, the array must be rearranged as the lowest possible order (i.e., sorted in ascending order).


Given an array of integers nums, find the next permutation of nums.

The replacement must be in place and use only constant extra memory.



In [15]:
def next_permutation(nums):
    # Find the first index 'i' from the end where nums[i] < nums[i+1]
    i = len(nums) - 2
    while i >= 0 and nums[i] >= nums[i + 1]:
        i -= 1

    if i >= 0:
        # Find index 'j' such that nums[j] is the next larger element than nums[i]
        j = len(nums) - 1
        while nums[j] <= nums[i]:
            j -= 1
        # Swap them
        nums[i], nums[j] = nums[j], nums[i]

    # Reverse the subarray from i+1 to the end in place using two pointers
    left, right = i + 1, len(nums) - 1
    while left < right:
        nums[left], nums[right] = nums[right], nums[left]
        left += 1
        right -= 1

# Example usage
nums = [1, 3, 2]
next_permutation(nums)
print(nums)  # Expected Output: [2, 1, 3]

[2, 1, 3]


In [14]:
nums = [1,2,3]

if not nums[1:] == sorted(nums[1:], reverse=True):
    nums[1:] = sorted(nums[1:], reverse=True)
elif max(nums) != nums[0]:
    max_num = max(nums)
    nums.remove(max_num)
    sorted(nums, reverse=False)
    nums.insert(0, max_num)
else:
    min_num = min(nums)
    nums.remove(min_num)
    nums = [3,2]
    print(nums)
    sorted(nums, reverse=False)
    nums.insert(0, min_num)

nums

[1, 3, 2]

In [13]:
nums = [3,2]
sorted(nums, reverse=False)

[2, 3]

## Find First and Last Position of Element in Sorted Array


Given an array of integers nums sorted in non-decreasing order, find the starting and ending position of a given target value.

If target is not found in the array, return [-1, -1].

You must write an algorithm with O(log n) runtime complexity.

In [33]:
def search_range(nums, target):
    def find_bound(is_first):
        left, right = 0, len(nums) - 1
        bound = -1
        while left <= right:
            mid = (left + right) // 2
            if nums[mid] == target:
                bound = mid
                if is_first:
                    right = mid - 1
                else:
                    left = mid + 1
            elif nums[mid] < target:
                left = mid + 1
            else:
                right = mid - 1
        return bound

    left_bound = find_bound(True)
    if left_bound == -1:
        return [-1, -1]
    right_bound = find_bound(False)
    return [left_bound, right_bound]

result = search_range([5,7,7,8,8,10], target)
print(result)

3 3
3 2
[3, 4]


In [28]:
output = [-1, -1]
nums = []


left, right = 0, 0
target = 8
while right < len(nums) - 1:
    if nums[left] == target:
        right += 1
        output[0] = left
        print(left)
    else:
        left += 1
        right += 1
    if len(output) >  0 and nums[right] == target:
        output[1] = right


print(output)


[-1, -1]


In [35]:
matrix = [[0 for _ in range(9)] for _ in range(9)]
print(matrix)

[[0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0]]


In [42]:
def is_valid_sudoku(board):
    # Check rows
    for row in board:
        seen = set()
        for num in row:
            if num != '.':
                if num in seen:
                    return False
                seen.add(num)

    # Check columns
    for col in range(9):
        seen = set()
        for row in range(9):
            num = board[row][col]
            if num != '.':
                if num in seen:
                    return False
                seen.add(num)

    # Check each 3x3 sub-box
    for sub_row in range(0, 9, 3):
        for sub_col in range(0, 9, 3):
            seen = set()
            for r in range(sub_row, sub_row + 3):
                for c in range(sub_col, sub_col + 3):
                    num = board[r][c]
                    if num != '.':
                        if num in seen:
                            return False
                        seen.add(num)

    return True


board = [["5","3",".",".","7",".",".",".","."]
,["6",".",".","1","9","5",".",".","."]
,[".","9","8",".",".",".",".","6","."]
,["8",".",".",".","6",".",".",".","3"]
,["4",".",".","8",".","3",".",".","1"]
,["7",".",".",".","2",".",".",".","6"]
,[".","6",".",".",".",".","2","8","."]
,[".",".",".","4","1","9",".",".","5"]
,[".",".",".",".","8",".",".","7","9"]]

print(is_valid_sudoku(board))

True


In [13]:


intervals.sort(key=lambda x: x[0])

print(intervals)

for i in range(len(intervals) - 1):
    print(i)
    if intervals[i][1] > intervals[i+1][0]:
        print(intervals[i][1])


[[0, 30], [5, 10], [15, 20]]
0
30
1


## Find the Maximum Length of Valid Subsequence II

You are given an integer array nums and a positive integer k. subsequence sub of nums with length x is called valid if it satisfies:

(sub[0] + sub[1]) % k == (sub[1] + sub[2]) % k == ... == (sub[x - 2] + sub[x - 1]) % k.
Return the length of the longest valid subsequence of nums.

In [149]:
nums = [1,4,2,3,1,4]
k = 3
count = {}

for i in range(len(nums)-1):
    print(i, i+1)
    count[(nums[i] + nums[i+ 1]) % k] = count.get((nums[i] + nums[i+ 1]) % k, 0) + 1



0 1
1 2
2 3
3 4
4 5


In [150]:
count

{2: 3, 0: 1, 1: 1}

## Missing ranges

A number x is considered missing if x is in the range [lower, upper] and x is not in nums.

In [186]:
def findMissingRanges(nums, lower, upper):
    """
    Given an inclusive range [lower, upper] and a sorted unique integer array nums,
    returns the shortest sorted list of ranges that exactly covers all the missing numbers.
    
    Approach:
      1. Set a 'prev' pointer to lower - 1.
      2. Append upper+1 to the end of nums to handle the gap between the last element and upper.
      3. For each current number in nums (plus the appended upper+1):
           - If there's a gap between prev and current (i.e. curr - prev >= 2), then
             the missing range is from prev+1 to curr-1. Format this range accordingly.
           - Update prev = current.
      4. Return the list of missing ranges.
    """
    missing = []
    prev = lower - 1
    for curr in nums + [upper + 1]:
        if curr - prev >= 2:
            missing.append([prev + 1, curr - 1])
        prev = curr
    return missing

In [187]:
# Example 1:
nums = [0, 1, 3, 50, 75]
lower = 0
upper = 99
# Missing numbers and ranges:
#   2
#   4->49
#   51->74
#   76->99
print("Missing Ranges:", findMissingRanges(nums, lower, upper))

Missing Ranges: [[2, 2], [4, 49], [51, 74], [76, 99]]


In [ ]:
def find_subarrays_intervals(A):
    """
    Given an array A of N integers, finds all subarrays (as [start, end] indices, 1-indexed)
    such that the average of the subarray is greater than the average of the remaining elements.
    
    For a subarray A[i...j] (0-indexed) with length L = j - i + 1 and sum S,
    and total sum T over N elements:
      - If L < N, then we require:  S / L > (T - S) / (N - L)
        This is equivalent to: S * N > T * L   (using strict inequality)
      - If L == N (i.e. the entire array), include it by default.
    
    Returns the list of intervals [start, end] (1-indexed) in sorted order (by start then end).
    """
    N = len(A)
    if N == 0:
        return []
    
    total = sum(A)
    # Build prefix sum array: prefix[i+1] = sum(A[0...i])
    prefix = [0] * (N + 1)
    for i in range(N):
        prefix[i+1] = prefix[i] + A[i]
    
    intervals = []
    for i in range(N):
        for j in range(i, N):
            L = j - i + 1
            S = prefix[j+1] - prefix[i]
            # For the entire array, include by default.
            if L == N:
                intervals.append([i+1, j+1])
            else:
                # Check condition: S * N > total * L
                if S * N > total * L:
                    intervals.append([i+1, j+1])
    # The intervals are generated in lexicographical order by (i, j).
    return intervals


# Example usage:
if __name__ == "__main__":
    A = [3, 4, 2]  # Given array
    result = find_subarrays_intervals(A)
    print("Subarrays with average greater than remaining:")
    # Expected output: [[1, 2], [1, 3], [2, 2]]
    print(result)

In [194]:
def jump(nums: list[int]) -> int:
    n = len(nums)
    jumps = 0        # number of jumps made
    current_end = 0  # end of current jump's range
    farthest = 0     # farthest index reachable
    
    # We iterate until the second last element because once we reach or pass it, 
    # we know that we have reached the end.
    for i in range(n - 1):
        farthest = max(farthest, i + nums[i])
        print(i, current_end)
        if i == current_end:  # reached the end of the current jump's range
            jumps += 1
            current_end = farthest
    return jumps

# Example usage:
if __name__ == "__main__":
    nums = [2, 3, 1, 1, 4]
    print(jump(nums))  # Expected output: 2

0 0
1 2
2 2
3 4
2


In [195]:
def exist(board: list[list[str]], word: str) -> bool:
    if not board or not board[0]:
        return False

    rows, cols = len(board), len(board[0])
    
    def dfs(i: int, j: int, pos: int) -> bool:
        # If all characters are found
        if pos == len(word):
            return True
        # Out-of-bounds or current cell doesn't match the current word character
        if i < 0 or i >= rows or j < 0 or j >= cols or board[i][j] != word[pos]:
            return False
        
        # Mark the current cell as visited by temporary replacing the value
        temp = board[i][j]
        board[i][j] = '#'
        
        # Explore all four directions: down, up, right, left
        for di, dj in [(1, 0), (-1, 0), (0, 1), (0, -1)]:
            if dfs(i + di, j + dj, pos + 1):
                return True
        
        # Backtrack: restore the current cell's value
        board[i][j] = temp
        return False
            
    for i in range(rows):
        for j in range(cols):
            if board[i][j] == word[0] and dfs(i, j, 0):
                return True
    return False

# Example usage:
if __name__ == "__main__":
    board = [
        ["A", "B", "C", "E"],
        ["S", "F", "C", "S"],
        ["A", "D", "E", "E"]
    ]
    word = "ABCCED"  # Expected output: True
    print(exist(board, word))

1 0
-1 0
0 1
1 0
-1 0
0 1
1 0
1 0
1 0
-1 0
0 1
0 -1
1 0
True


In [ ]:
def minRemoveToMakeValid(s: str) -> str:
    # First pass: use a stack to track indices of '(' that haven't found a match.
    stack = []
    to_remove = set()  # Set to record indices of unmatched parentheses.
    
    for i, ch in enumerate(s):
        if ch == '(':
            # Record the index of the opening parenthesis.
            stack.append(i)
        elif ch == ')':
            if stack:
                # There is a matching '(' in the stack, so one pair is formed.
                stack.pop()
            else:
                # No matching '(' is available, so mark this ')' for removal.
                to_remove.add(i)
    
    # After traversing the string, any '(' indices still left in the stack did not have a matching ')'
    # Mark all these indices for removal.


    
    print(to_remove)
    # Build the resulting string by iterating over the original string and
    # including only characters whose indices are not marked in the to_remove set.
    result = []
    for i, ch in enumerate(s):
        if i not in to_remove:
            result.append(ch)
            
    return ''.join(result)

# Example usage:
if __name__ == "__main__":
    s = "(("  # Example string where some parentheses might be unmatched.
    print(minRemoveToMakeValid(s))  # Expected output: "ab(c)d" or any valid variant.

{0, 1}



In [8]:
count

6